# Flight Agent: Your Travel Planner

Time for an exciting use case. Now is the time to use all the knowledge we have gathered so far to build a complete AI Agent ourselves.

## Plan of Attack:

1. Import libraries
2. Define our tools
3. Define our tools schema
4. Call the OpenAI Responses API
5. Handle tool calls
6. Chat logic
7. Gradio Interface
8. Deploy to Huggingface spaces

## Step 1: Import libraries

In [1]:
import os
from tavily import TavilyClient
from dotenv import load_dotenv
import json
from openai import OpenAI
from utils import function_to_tool
from IPython.display import display, Markdown
import gradio as gr

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if not TAVILY_API_KEY:
    raise ValueError("TAVILY_API_KEY is not set in the environment variables.")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

tavily_client = TavilyClient()
openai_client = OpenAI()

## Step 2: Define our tools

In [2]:
# Flight search tool

query = "I want to travel to Tokyo and Osaka tomorrow. What are the best flights available?"

response = tavily_client.search(
    query=query,
    include_domains=['skyscanner.com', 'expedia.com', 'booking.com']
)

print("Search results: \n", json.dumps(response, indent=2))


Search results: 
 {
  "query": "I want to travel to Tokyo and Osaka tomorrow. What are the best flights available?",
  "follow_up_questions": null,
  "answer": null,
  "images": [],
  "results": [
    {
      "url": "https://www.expedia.com/Destinations-In-Japan.d89.Flight-Destinations",
      "title": "$258 Cheap Flights & Airfare to Japan - Expedia.com",
      "content": "How much is the cheapest flight to Japan? Prices were available within the past 7 days and start at $258 for one-way flights and $532 for round trip,",
      "score": 0.98571,
      "raw_content": null
    },
    {
      "url": "https://www.expedia.com/Cheap-Flights-To-Tokyo.d179900.Travel-Guide-Flights",
      "title": "Cheap Flights to Tokyo - Expedia",
      "content": "The lowest price for round-trip flights to Tokyo in 2025 starts from from $532, while one-way tickets start from from $258.",
      "score": 0.98281,
      "raw_content": null
    },
    {
      "url": "https://www.skyscanner.com/routes/osaa/nrt/o

In [2]:
# create a function
def flight_search(query: str) -> list:
    """
    Search for flights using the Tavily API.

    Args:
        query (str): The user's flight search query.

    Returns:
        list: The search results from the flight API.
    """
    
    response = tavily_client.search(
        query=query,
        include_domains=['skyscanner.com', 'expedia.com', 'booking.com']
    )

    results = []

    for result in response["results"]:
        results.append(result["content"])

    return results

In [3]:
def hotel_search(query: str) -> list:
    """
    Search for hotels using the Tavily API.

    Args:
        query (str): The user's hotel search query.

    Returns:
        list: The search results from the hotel API.
    """
    response = tavily_client.search(
        query=query,
        include_domains=['booking.com', 'airbnb.com', 'hotels.com']
    )

    results = []

    for result in response["results"]:
        results.append(result["content"])

    return results

## Step 3: Define our tools schema

In [4]:
flight_tool_schema = function_to_tool(flight_search)

In [5]:
hotel_tool_schema = function_to_tool(hotel_search)

In [6]:
tools = [flight_tool_schema, hotel_tool_schema]

In [7]:
flight_tool_schema

{'type': 'function',
 'name': 'flight_search',
 'description': 'Search for flights using the Tavily API.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string',
    'description': "The user's flight search query."}},
  'required': ['query']}}

## Step 4: Call the OpenAI Responses API

In [8]:
system_message = """

You are a helpful travel planner and your are to assist the user in planning their travel itinerary.

You are to ask the user in a conversational manner about their travel plans and preferences, first start with their
destination, wait for their response and then ask about the dates they plan to travel, and then search for the best flights available,
build an itinerary based on their preferences and give them a summary of their travel plans.


Eg.
- Hello, I am your travel planner. Which country would you like to travel to?
- I want to travel to Japan
- What is your preferred destination in Japan?
- Tokyo
- Great! When do you plan to travel?
- I plan to travel tomorrow.

* use your search tool to search for flight deals *
* Wait until you are provided with flight options *
* use your search tool to search for hotel booking deals *
* Wait until you are provided the results of the hotel search *
* summarize results in a neatly formatted itinerary. *
* provide the itinerary to the user *

- Here is your travel itinerary for Japan:

Remember to only provide the user with the results once you have information about the flights as well as the hotels.
"""

In [19]:
def chat(input_list, tools):
    response = openai_client.responses.create(
        model="gpt-4o-mini",
        input=input_list,
        tools=tools,
        tool_choice="auto",
        parallel_tool_calls=False
    )

    if response.output[0].type == "message":
        input_list.append({"role": "assistant", "content": response.output_text})
    if response.output[0].type == "function_call":
        input_list += response.output

    return response

In [20]:
input_list = [{"role": "system", "content": system_message}]

In [29]:
input_list

[{'role': 'system',
  'content': '\n\nYou are a helpful travel planner and your are to assist the user in planning their travel itinerary.\n\nYou are to ask the user in a conversational manner about their travel plans and preferences, first start with their\ndestination, wait for their response and then ask about the dates they plan to travel, and then search for the best flights available,\nbuild an itinerary based on their preferences and give them a summary of their travel plans.\n\n\nEg.\n- Hello, I am your travel planner. Which country would you like to travel to?\n- I want to travel to Japan\n- What is your preferred destination in Japan?\n- Tokyo\n- Great! When do you plan to travel?\n- I plan to travel tomorrow.\n\n* use your search tool to search for flight deals *\n* Wait until you are provided with flight options *\n* use your search tool to search for hotel booking deals *\n* Wait until you are provided the results of the hotel search *\n* summarize results in a neatly form

In [28]:
user_message = "Tomorrow"
input_list.append({"role": "user", "content": user_message})

response = chat(input_list, tools)

print(response.output)

[ResponseFunctionToolCall(arguments='{"query":"flights to Tokyo tomorrow"}', call_id='call_CRZlnJ6o8io91SZ1yKAfrpaK', name='flight_search', type='function_call', id='fc_68ac13e20b2481a196d8bb5b9f5d506000ddf42853264a78', status='completed')]


## Step 5: Handle tools calls

In [30]:
# grab the function name and arguments from LLM response

name = response.output[0].name
args = json.loads(response.output[0].arguments)

In [31]:
print("Name of function: ", name)
print("Arguments: ", args)

Name of function:  flight_search
Arguments:  {'query': 'flights to Tokyo tomorrow'}


In [19]:
# call the function
def call_function(name, args):
    if name == "flight_search":
        return flight_search(**args)
    if name == "hotel_search":
        return hotel_search(**args)

In [34]:
results = call_function(name, args)

In [35]:
# transform the response to message list format
input_list.append({
    "type": "function_call_output",
    "call_id": response.output[0].call_id,
    "output": str(results)
})

In [36]:
input_list

[{'role': 'system',
  'content': '\n\nYou are a helpful travel planner and your are to assist the user in planning their travel itinerary.\n\nYou are to ask the user in a conversational manner about their travel plans and preferences, first start with their\ndestination, wait for their response and then ask about the dates they plan to travel, and then search for the best flights available,\nbuild an itinerary based on their preferences and give them a summary of their travel plans.\n\n\nEg.\n- Hello, I am your travel planner. Which country would you like to travel to?\n- I want to travel to Japan\n- What is your preferred destination in Japan?\n- Tokyo\n- Great! When do you plan to travel?\n- I plan to travel tomorrow.\n\n* use your search tool to search for flight deals *\n* Wait until you are provided with flight options *\n* use your search tool to search for hotel booking deals *\n* Wait until you are provided the results of the hotel search *\n* summarize results in a neatly form

In [37]:
# provide update message list to LLM

response = chat(input_list, tools)
print(response.output)

[ResponseOutputMessage(id='msg_68ac1d3a202481a1a44f4509e8bbe45100ddf42853264a78', content=[ResponseOutputText(annotations=[], text="I found some flight options for you! Here are the details:\n\n### Flights to Tokyo\n- **From Los Angeles (LAX)**: \n  - One-way: Starting at **$258**\n  - Round-trip: Starting at **$532**\n\n- **From New York (JFK)**: \n  - One-way: Starting at **$432**\n  - Round-trip: Starting at **$799**\n\nNow, let's find accommodation for your stay in Tokyo. How many nights will you be staying?", type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]


In [38]:
display(Markdown(response.output_text))

I found some flight options for you! Here are the details:

### Flights to Tokyo
- **From Los Angeles (LAX)**: 
  - One-way: Starting at **$258**
  - Round-trip: Starting at **$532**

- **From New York (JFK)**: 
  - One-way: Starting at **$432**
  - Round-trip: Starting at **$799**

Now, let's find accommodation for your stay in Tokyo. How many nights will you be staying?

## Step 6: Gradio UI

In [9]:
def get_response(input_list, tools):
    response = openai_client.responses.create(
        model="gpt-4o-mini",
        input=input_list,
        tools=tools,
        tool_choice="auto",
        parallel_tool_calls=False
    )
    return response

In [14]:
def clean_history(history):
    return [
        {"role": msg["role"], "content": msg["content"]}
        for msg in history
        if "role" in msg and "content" in msg
    ]

In [17]:
def chat(message, history):

    history = clean_history(history)

    input_list = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # print(input_list)
    response = get_response(input_list, tools)

    if response.output[0].type == "function_call":
        while response.output[0].type == "function_call":
            input_list += response.output

            name = response.output[0].name
            args = json.loads(response.output[0].arguments)

            results = call_function(name, args)

            input_list.append({
                "type": "function_call_output",
                "call_id": response.output[0].call_id,
                "output": str(results)
            })

            response = get_response(input_list, tools)

    return response.output_text

In [20]:
view = gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
